# Introduction to the Keras Tuner

Install and import the Keras Tuner.

In [1]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

## Download and prepare the dataset

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Define the model


In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning



In [6]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

Create a callback to stop training early after reaching a certain value for the validation loss.

In [7]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [8]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 23s]
val_accuracy: 0.8785833120346069

Best val_accuracy So Far: 0.8927500247955322
Total elapsed time: 00h 14m 02s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 224 and the optimal learning rate for the optimizer
is 0.001.



## Train the model

In [9]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5043 - accuracy: 0.8210 - val_loss: 0.4480 - val_accuracy: 0.8393
Epoch 2/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3801 - accuracy: 0.8616 - val_loss: 0.3632 - val_accuracy: 0.8686
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3378 - accuracy: 0.8770 - val_loss: 0.3502 - val_accuracy: 0.8710
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3122 - accuracy: 0.8831 - val_loss: 0.3271 - val_accuracy: 0.8825
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2919 - accuracy: 0.8918 - val_loss: 0.3261 - val_accuracy: 0.8840
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2773 - accuracy: 0.8967 - val_loss: 0.3223 - val_accuracy: 0.8866
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2619 - accuracy: 0.9029 - val_loss: 0.3197 - val_accuracy:

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [10]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/23
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5052 - accuracy: 0.8227 - val_loss: 0.4198 - val_accuracy: 0.8482
Epoch 2/23
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3758 - accuracy: 0.8636 - val_loss: 0.3737 - val_accuracy: 0.8648
Epoch 3/23
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3345 - accuracy: 0.8779 - val_loss: 0.3598 - val_accuracy: 0.8743
Epoch 4/23
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3105 - accuracy: 0.8845 - val_loss: 0.3418 - val_accuracy: 0.8734
Epoch 5/23
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2896 - accuracy: 0.8918 - val_loss: 0.3348 - val_accuracy: 0.8832
Epoch 6/23
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2740 - accuracy: 0.8982 - val_loss: 0.3357 - val_accuracy: 0.8798
Epoch 7/23
1500/1500 [==============================] - 5s 4ms/step - loss: 0.2613 - accuracy: 0.9032 - val_loss: 0.3329 - val_accuracy:

### evaluate the hypermodel on the test data.

In [11]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 4ms/step - loss: 0.4351 - accuracy: 0.8813
[test loss, test accuracy]: [0.4350650906562805, 0.8812999725341797]
